In [1]:
import json
import datetime

from bs4 import BeautifulSoup
import requests
import functools

In [2]:
url = "https://elpais.com/espana/madrid/"

response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
else:
    print("Error: Unable to fetch the webpage.")

In [3]:
soup = BeautifulSoup(html_content, "lxml")

In [4]:
@functools.cache
def get_description(link_noticia: str):
    response = requests.get(link_noticia)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, "lxml")
        description = soup.find("h2", attrs={"class": "a_st"})
        return description.text.strip()
    else:
        return "Error: Unable to fetch the webpage."

In [5]:
datetime.datetime.now().strftime("%Y-%m-%d")

'2024-04-03'

In [6]:
titles = soup.find_all("h2", attrs={"class": "c_t"})

data = []
today = datetime.datetime.now().strftime("%Y-%m-%d")
for title in titles:
    link = title.find("a")["href"]
    # Example link:
    # https://elpais.com/espana/madrid/2024-04-02/nombre-noticia.html
    date_from_link = link.split("/")[-2]
    if date_from_link != today:
        continue
    noticia = {}
    noticia["title"] = title.text.strip()
    noticia["link"] = link
    noticia["description"] = get_description(noticia["link"])
    data.append(noticia)
data

[{'title': 'Así vetó el PP de Ayuso que MAR diera la cara en la Asamblea por sus presiones a periodistas: “[Es] una reprobación encubierta con juicio de valor”',
  'link': 'https://elpais.com/espana/madrid/2024-04-03/asi-veto-el-pp-de-ayuso-que-mar-diera-la-cara-en-la-asamblea-por-sus-presiones-a-periodistas-es-una-reprobacion-encubierta-con-juicio-de-valor.html',
  'description': 'EL PAÍS reconstruye la reunión en la que el PP evitó las comparecencias del alto cargo, una decisión que la oposición está dispuesta a llevar hasta el Constitucional'},
 {'title': 'Quince heridos al chocar un autobús municipal contra un muro en Valdemoro',
  'link': 'https://elpais.com/espana/madrid/2024-04-03/quince-heridos-al-chocar-un-autobus-municipal-contra-un-muro-en-valdemoro.html',
  'description': 'De las víctimas, las más graves son una pasajera de 62 años y la conductora, de 58'}]

In [7]:
with open(
    f"noticias_{datetime.datetime.now().strftime('%Y-%m-%d')}.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

In [8]:
with open("noticias_2024-04-03.json", "r", encoding="utf-8") as f:
    load_data = json.load(f)

In [9]:
load_data

[{'title': 'Así vetó el PP de Ayuso que MAR diera la cara en la Asamblea por sus presiones a periodistas: “[Es] una reprobación encubierta con juicio de valor”',
  'link': 'https://elpais.com/espana/madrid/2024-04-03/asi-veto-el-pp-de-ayuso-que-mar-diera-la-cara-en-la-asamblea-por-sus-presiones-a-periodistas-es-una-reprobacion-encubierta-con-juicio-de-valor.html',
  'description': 'EL PAÍS reconstruye la reunión en la que el PP evitó las comparecencias del alto cargo, una decisión que la oposición está dispuesta a llevar hasta el Constitucional'},
 {'title': 'Quince heridos al chocar un autobús municipal contra un muro en Valdemoro',
  'link': 'https://elpais.com/espana/madrid/2024-04-03/quince-heridos-al-chocar-un-autobus-municipal-contra-un-muro-en-valdemoro.html',
  'description': 'De las víctimas, las más graves son una pasajera de 62 años y la conductora, de 58'}]